In [1]:
import pandas as pd
import time
import json
from bs4 import BeautifulSoup
from common.crawl_func import *
from common.scrap_func import *
from common.batch_util import set_headinfo,get_headinfo
from common.log_util import *
from common.util import json_loads,find_between,find_between,fill_list
from common.crawling_util import simple_crawling,session_crawling,jsonpayload_crawling, payload_crawling
logger_initialize('dev_logger_setting.json')

In [187]:
def crawling_OZ_dom(dpt,arr,dpt_date):
    ##출발지, 도착지, 출발일을 기준으로 국내선(국제선) 편도 가격 읽어오기
    print('Crawling asiana homepage schedule site')
    url = "https://flyasiana.com/I/ko/RevenueDomesticFareDrivenFlightSelect.do"
    head = {
        'Referer':'https://flyasiana.com/I/ko/RevenueDomesticTravelRegist.do',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }
    param ={
        'hidPageType':'',      'departureAirportC':'',
        'arrivalAirportC':'',  'departureDateC':'',
        'arrivalDateC':'',     'officeId':'',
        'entType':'',          'corporateCode':'',
        'couponDesc':'',       'cServiceType':'',
        'couponCode':'',       'query':'',
        'CallPage':'RevenueDomesticTravelRegist',
        #'sessionUniqueKey':'ec5fd537-302d-48f0-fee5-f4b649026ecf',
        'domIntType':'D',               'tripType':'OW',
        'departureAirport':dpt,       'arrivalAirport':arr,
        'openDepartureAirport1':dpt,  'openArrivalAirport1':arr,
        'openDepartureAirport2':arr,  'openArrivalAirport2':dpt,
        'departureDate':dpt_date,   'arrivalDate':dpt_date,
        '__strSDate':dpt_date,      '__strEDate':dpt_date,
        'adultCount':'1',      'childCount':'0',
        'infantCount':'0',     'ageCalYear':'2017',
        'ageCalMonth':'1',     'ageCalDay':'0',    
    }

    raw_text = simple_crawling(url,param,head=head,method='post',json=False)
    start_txt = "var depFareFamilyWithAllAvail = eval('"
    end_txt = "');"
    return find_between(raw_text,start_txt,end_txt)

In [190]:
## 포맷 : 대상항공사코드,플라이트,출발일,출발지,도착지,출발시간,도착시간,가격,유류세,세금,좌석수
def scraping_OZ_dom_int(head,raw_data):
    if raw_data is None or raw_data == '':
        return None
    ## json 형태로 변경 처리
    json_data = {'json_data':eval(raw_data.replace('true',"True").replace('false',"False"))}
    sch_list = []
    for line_data in json_data['json_data']:
        ## fare 정보 찾아
        #fare_data = line_data['fareFamilyAmounts'][0]['itineraryAvailDataOfRecommend']['paxTypeFareDatas'][0]
        ## flight 정보 찾아오기
        for family_info in line_data['fareFamilyAmounts']:
            fare_data = family_info['itineraryAvailDataOfRecommend']['paxTypeFareDatas'][0]
            for flt_info in family_info['itineraryAvailDataOfRecommend']['availDataList'][0]:
                sch = []
                flt_data = flt_info['flightInfoDatas'][0]
                ## flight 정보 추가
                sch.extend([flt_data['carrierCode'],flt_data['flightNo'],flt_data['departureDate'][:8],
                            flt_data['departureAirport'],flt_data['arrivalAirport'],
                            flt_data['departureDate'][8:12],flt_data['arrivalDate'][8:12]])
                ## fare 정보 추가
                sch.extend([fare_data['amountWithoutTax'],fare_data['fuelCharge'],fare_data['totalTax']])
                ## seat 정보 추가
                sch.append(flt_info['seatCount'] if flt_info['seatCount'] != '' else '0')
                sch_list.append(sch)
    return sch_list


In [191]:
dpt,arr,dpt_date = 'GMP','CJU','20170820'
raw_data = crawling_OZ_dom(dpt,arr,dpt_date)
raw_list = scraping_OZ_dom_int('',raw_data)
pd.DataFrame(sorted(raw_list))

Crawling asiana homepage schedule site
Start Simple crawling :  https://flyasiana.com/I/ko/RevenueDomesticFareDrivenFlightSelect.do
>> Parameters
arrivalDate:20170820 , couponDesc: , CallPage:RevenueDomesticTravelRegist , openDepartureAirport1:GMP , openArrivalAirport2:GMP , entType: , adultCount:1 , __strSDate:20170820 , ageCalYear:2017 , domIntType:D , openArrivalAirport1:CJU , infantCount:0 , officeId: , arrivalAirport:CJU , childCount:0 , departureAirportC: , couponCode: , departureDateC: , __strEDate:20170820 , departureAirport:GMP , ageCalMonth:1 , arrivalDateC: , openDepartureAirport2:CJU , cServiceType: , hidPageType: , tripType:OW , corporateCode: , ageCalDay:0 , arrivalAirportC: , query: , departureDate:20170820 , 
End Simple crawling


,0,1,2,3,4,5,6,7,8,9,10
0,OZ,8901,20170820,GMP,CJU,0610,0720,113000.00,2200.00,6200.00,9
1,OZ,8901,20170820,GMP,CJU,0610,0720,68000.00,2200.00,6200.00,2
2,OZ,8903,20170820,GMP,CJU,0625,0730,113000.00,2200.00,6200.00,1
3,OZ,8907,20170820,GMP,CJU,0710,0820,113000.00,2200.00,6200.00,5
4,OZ,8913,20170820,GMP,CJU,0845,0955,113000.00,2200.00,6200.00,1
5,OZ,8923,20170820,GMP,CJU,1950,2100,113000.00,2200.00,6200.00,2
6,OZ,8923,20170820,GMP,CJU,1950,2100,51000.00,2200.00,6200.00,2
7,OZ,8923,20170820,GMP,CJU,1950,2100,57000.00,2200.00,6200.00,2
8,OZ,8931,20170820,GMP,CJU,1355,1505,113000.00,2200.00,6200.00,3
9,OZ,8937,20170820,GMP,CJU,1815,1925,113000.00,2200.00,6200.00,9


In [164]:
datas = eval(raw_data.replace('true',"True").replace('false',"False"))
datas
#datas[0]['fareFamilyAmounts'][1]['itineraryAvailDataOfRecommend']['paxTypeFareDatas']#[0]

[{'addMileageRatio': '100',
  'brandName': '할인운임',
  'currency': 'KRW',
  'currencySymbol': '',
  'fareFamily': 'KRJP1HEQ',
  'fareFamilyAmounts': [{'adtDiscountRate': '0',
    'amount': '292000.00',
    'amountWithoutTax': '264000.00',
    'chdDiscountRate': '0',
    'currency': 'KRW',
    'fuelCharge': '0.00',
    'itineraryAvailDataOfRecommend': {'availDataList': [[{'allowTicketing': True,
        'arrivalAirport': 'NRT',
        'arrivalAirportDesc': '도쿄/나리타',
        'arrivalCity': '',
        'arrivalCityDesc': '',
        'arrivalDate': '20170730111000',
        'bookingClass': 'E',
        'canSelectedAvailDatas': [],
        'departureAirport': 'ICN',
        'departureAirportDesc': '인천',
        'departureCity': '',
        'departureCityDesc': '',
        'departureDate': '20170730090000',
        'expectAddMileage': '0',
        'fareFamily': 'KRJP1HEQ',
        'fareFamilyDesc': 'Brand name not yet defined',
        'fixedFlight': False,
        'flightId': '0',
        'f

In [149]:
datas = eval(raw_data.replace('true',"True").replace('false',"False"))
#len(datas)
datas[0]['fareFamilyAmounts'][1]['itineraryAvailDataOfRecommend']['availDataList'][0]#[0]#['flightInfoDatas']

[{'allowTicketing': True,
  'arrivalAirport': 'NRT',
  'arrivalAirportDesc': '도쿄/나리타',
  'arrivalCity': '',
  'arrivalCityDesc': '',
  'arrivalDate': '20170730183000',
  'bookingClass': 'H',
  'canSelectedAvailDatas': [],
  'departureAirport': 'ICN',
  'departureAirportDesc': '인천',
  'departureCity': '',
  'departureCityDesc': '',
  'departureDate': '20170730161000',
  'expectAddMileage': '0',
  'fareFamily': 'KRJP1HEQ',
  'fareFamilyDesc': 'Brand name not yet defined',
  'fixedFlight': False,
  'flightId': '3',
  'flightInfoDatas': [{'aircraftType': '321',
    'aircraftTypeDesc': 'Airbus Industrie A321',
    'allowTicketing': True,
    'alteaLineNumber': '',
    'arrivalAirport': 'NRT',
    'arrivalAirportDesc': '도쿄/나리타',
    'arrivalCity': '',
    'arrivalCityDesc': '',
    'arrivalCountry': '',
    'arrivalDate': '20170730183000',
    'arrivalDateOfSchedule': '',
    'bDayIndicator': '',
    'bookingClass': '',
    'cabinClass': '',
    'cabinClassName': '',
    'carrierCode': 'OZ',

In [137]:
raw_data

'[{"addMileageRatio":"100","brandName":"할인운임","currency":"KRW","currencySymbol":"","fareFamily":"KRJP1HEQ","fareFamilyAmounts":[{"adtDiscountRate":"0","amount":"292000.00","amountWithoutTax":"264000.00","chdDiscountRate":"0","currency":"KRW","fuelCharge":"0.00","itineraryAvailDataOfRecommend":{"availDataList":[[{"allowTicketing":true,"arrivalAirport":"NRT","arrivalAirportDesc":"도쿄/나리타","arrivalCity":"","arrivalCityDesc":"","arrivalDate":"20170730111000","bookingClass":"E","canSelectedAvailDatas":[],"departureAirport":"ICN","departureAirportDesc":"인천","departureCity":"","departureCityDesc":"","departureDate":"20170730090000","expectAddMileage":"0","fareFamily":"KRJP1HEQ","fareFamilyDesc":"Brand name not yet defined","fixedFlight":false,"flightId":"0","flightInfoDatas":[{"aircraftType":"333","aircraftTypeDesc":"Airbus Industrie A330-300","allowTicketing":true,"alteaLineNumber":"","arrivalAirport":"NRT","arrivalAirportDesc":"도쿄/나리타","arrivalCity":"","arrivalCityDesc":"","arrivalCountry":"

In [142]:
scraping_OZ_dom('',raw_data)

[['OZ',
  '102',
  '20170730',
  'ICN',
  'NRT',
  '0900',
  '1110',
  '264000.00',
  '0.00',
  '28000.00',
  '9'],
 ['OZ',
  '104',
  '20170730',
  'ICN',
  'NRT',
  '1000',
  '1220',
  '264000.00',
  '0.00',
  '28000.00',
  '4'],
 ['OZ',
  '108',
  '20170730',
  'ICN',
  'NRT',
  '1850',
  '2110',
  '264000.00',
  '0.00',
  '28000.00',
  '3'],
 ['OZ',
  '102',
  '20170730',
  'ICN',
  'NRT',
  '0900',
  '1110',
  '313200.00',
  '0.00',
  '28000.00',
  '9'],
 ['OZ',
  '104',
  '20170730',
  'ICN',
  'NRT',
  '1000',
  '1220',
  '313200.00',
  '0.00',
  '28000.00',
  '8'],
 ['OZ',
  '106',
  '20170730',
  'ICN',
  'NRT',
  '1610',
  '1830',
  '313200.00',
  '0.00',
  '28000.00',
  '4'],
 ['OZ',
  '108',
  '20170730',
  'ICN',
  'NRT',
  '1850',
  '2110',
  '313200.00',
  '0.00',
  '28000.00',
  '8']]

In [106]:
site = 'OZ'
head = get_headinfo(set_headinfo(site,'1',site,dpt,arr,dpt_date,crawl_date=None))
raw_list = scraping_OZ_dom(head,raw_data)
pd.DataFrame(raw_list)

,0,1,2,3,4,5,6,7,8,9,10
0,OZ,8997,20170730,GMP,CJU,1750,1900,91000.00,2200.00,6200.00,9
1,OZ,8995,20170730,GMP,CJU,1410,1520,102000.00,2200.00,6200.00,9
2,OZ,8975,20170730,GMP,CJU,1425,1530,102000.00,2200.00,6200.00,9
3,OZ,8965,20170730,GMP,CJU,1455,1605,102000.00,2200.00,6200.00,9
4,OZ,8977,20170730,GMP,CJU,1500,1610,102000.00,2200.00,6200.00,9
5,OZ,8939,20170730,GMP,CJU,1525,1635,102000.00,2200.00,6200.00,9
6,OZ,8945,20170730,GMP,CJU,1610,1720,102000.00,2200.00,6200.00,9
7,OZ,8947,20170730,GMP,CJU,1620,1730,102000.00,2200.00,6200.00,9
8,OZ,8969,20170730,GMP,CJU,1745,1855,102000.00,2200.00,6200.00,9
9,OZ,8937,20170730,GMP,CJU,1815,1925,102000.00,2200.00,6200.00,9
